In [ ]:
"""
Delta Lake - Teste Completo dos Principais Recursos
Versão testada e funcionando!
"""

from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import col, lit


In [ ]:
print("=" * 70)
print("DELTA LAKE - TESTE COMPLETO")
print("=" * 70)

spark = SparkSession.builder \
    .appName("DeltaLakeCompleto") \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.13:3.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"\n✅ Spark {spark.version} iniciado!\n")

# Caminho da tabela
caminho = "/tmp/clientes_delta"

In [ ]:
# ========== 1. CRIAR TABELA DELTA ==========
print("📝 1. CRIANDO TABELA DELTA")
print("-" * 70)

clientes = spark.createDataFrame([
    (1, "João Silva", "SP", 5000.00, "Ativo"),
    (2, "Maria Santos", "RJ", 7500.00, "Ativo"),
    (3, "Pedro Costa", "MG", 4500.00, "Ativo"),
    (4, "Ana Souza", "SP", 6000.00, "Ativo"),
    (5, "Carlos Lima", "RS", 5500.00, "Ativo")
], ["id", "nome", "estado", "credito", "status"])

clientes.write.format("delta").mode("overwrite").save(caminho)
print("✅ Tabela criada com sucesso!")
print(f"   Localização: {caminho}\n")

spark.read.format("delta").load(caminho).show()


In [ ]:
# ========== 2. LER DADOS ==========
print("\n📖 2. LENDO DADOS DA TABELA")
print("-" * 70)

df = spark.read.format("delta").load(caminho)
print(f"Total de clientes: {df.count()}")
print(f"Clientes de SP: {df.filter(col('estado') == 'SP').count()}\n")


In [ ]:
# ========== 3. ADICIONAR NOVOS DADOS (APPEND) ==========
print("➕ 3. ADICIONANDO NOVOS CLIENTES")
print("-" * 70)

novos_clientes = spark.createDataFrame([
    (6, "Beatriz Alves", "BA", 6500.00, "Ativo"),
    (7, "Rafael Dias", "PE", 5800.00, "Ativo")
], ["id", "nome", "estado", "credito", "status"])

novos_clientes.write.format("delta").mode("append").save(caminho)
print("✅ Novos clientes adicionados!\n")

spark.read.format("delta").load(caminho).show()

In [ ]:
# ========== 4. ATUALIZAR DADOS (UPDATE) ==========
print("\n🔄 4. ATUALIZANDO DADOS")
print("-" * 70)
print("Aumentando crédito em 10% para clientes de SP...")

delta_table = DeltaTable.forPath(spark, caminho)

delta_table.update(
    condition = col("estado") == "SP",
    set = {"credito": col("credito") * 1.10}
)

print("✅ Créditos atualizados!\n")
spark.read.format("delta").load(caminho).orderBy("id").show()


In [ ]:
# ========== 5. DELETAR DADOS (DELETE) ==========
print("\n🗑️  5. REMOVENDO DADOS")
print("-" * 70)
print("Removendo cliente com id = 3...")

delta_table.delete(condition = col("id") == 3)

print("✅ Cliente removido!\n")
spark.read.format("delta").load(caminho).orderBy("id").show()


In [ ]:
# ========== 6. MERGE (UPSERT) ==========
print("\n🔀 6. MERGE (UPSERT)")
print("-" * 70)
print("Atualizando cliente 2 e inserindo cliente 8...")

dados_merge = spark.createDataFrame([
    (2, "Maria Santos Silva", "RJ", 8000.00, "Premium"),  # Atualizar
    (8, "Fernando Rocha", "SC", 7000.00, "Ativo")         # Inserir
], ["id", "nome", "estado", "credito", "status"])

delta_table.alias("target").merge(
    dados_merge.alias("source"),
    "target.id = source.id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

print("✅ MERGE executado!")
print("   - Cliente 2 atualizado (nome e crédito)")
print("   - Cliente 8 inserido\n")

spark.read.format("delta").load(caminho).orderBy("id").show()

In [ ]:
# ========== 7. HISTÓRICO DE VERSÕES ==========
print("\n📜 7. HISTÓRICO DE VERSÕES (TIME TRAVEL)")
print("-" * 70)

history = delta_table.history()
print("Todas as operações realizadas:")
history.select("version", "timestamp", "operation", "operationMetrics") \
    .show(truncate=False)


In [ ]:

# ========== 8. TIME TRAVEL - VERSÃO ANTERIOR ==========
print("\n⏰ 8. VIAJANDO NO TEMPO")
print("-" * 70)
print("Voltando para a versão 0 (dados originais)...\n")

df_v0 = spark.read \
    .format("delta") \
    .option("versionAsOf", 0) \
    .load(caminho)

print("Dados na versão 0:")
df_v0.show()

print("Comparação:")
print(f"  Versão 0: {df_v0.count()} clientes")
print(f"  Versão atual: {spark.read.format('delta').load(caminho).count()} clientes")

In [ ]:
# ========== 9. FILTROS E CONSULTAS ==========
print("\n🔍 9. CONSULTAS E FILTROS")
print("-" * 70)

df_atual = spark.read.format("delta").load(caminho)

print("Clientes com crédito > 6000:")
df_atual.filter(col("credito") > 6000).show()

print("Clientes por estado:")
df_atual.groupBy("estado").count().orderBy("estado").show()

print("Cliente com maior crédito:")
df_atual.orderBy(col("credito").desc()).limit(1).show()


In [ ]:
# ========== RESUMO FINAL ==========
print("\n" + "=" * 70)
print("✅ TESTE COMPLETO CONCLUÍDO COM SUCESSO!")
print("=" * 70)
print("\n📊 Operações testadas:")
print("  ✓ CREATE - Criar tabela Delta")
print("  ✓ READ - Ler dados")
print("  ✓ APPEND - Adicionar registros")
print("  ✓ UPDATE - Atualizar registros")
print("  ✓ DELETE - Remover registros")
print("  ✓ MERGE - Upsert (atualizar ou inserir)")
print("  ✓ HISTORY - Ver histórico de versões")
print("  ✓ TIME TRAVEL - Acessar versões anteriores")
print("  ✓ QUERY - Filtros e consultas")
print("\n💡 Delta Lake está 100% funcional!")
print(f"📁 Dados em: {caminho}")
print("=" * 70)

spark.stop()
print("\n🛑 Spark encerrado.")